# Project Code and Data Walkthrough

### Duncan Park and Kyle Parran

#### This notebook intends to give users a brief tour of the data pulled, processed, and analyzed in our project.



#




## Import Statements

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# local modules
import sys
sys.path.append('../src/')
from calc_format_futures_data import *
from pull_futures_data import *

import warnings
warnings.filterwarnings("ignore")

Loading library list...
Done


## Loading Data from WRDS

#### First, for each commodity you try to find info on all the futures contracts for that respective commodity

In [2]:
info_df = fetch_wrds_contract_info(2036, 'paper')
info_df

,futcode,contrcode,contrname,contrdate,startdate,lasttrddate
0,46.0,2036.0,ORANGE JUICE (FCOJ-A),0303,2001-10-08,2003-03-11
1,189.0,2036.0,ORANGE JUICE (FCOJ-A),0300,1998-09-08,2000-03-13
2,190.0,2036.0,ORANGE JUICE (FCOJ-A),0704,2002-12-03,2004-07-12
3,391.0,2036.0,ORANGE JUICE (FCOJ-A),0986,1985-04-01,1986-09-17
4,912.0,2036.0,ORANGE JUICE (FCOJ-A),0188,1986-08-01,1988-01-18
...,...,...,...,...,...,...
201,318247.0,2036.0,ORANGE JUICE (FCOJ-A),0978,1977-10-03,1978-09-18
202,318248.0,2036.0,ORANGE JUICE (FCOJ-A),1174,1973-12-03,1974-11-15
203,318249.0,2036.0,ORANGE JUICE (FCOJ-A),1175,1975-01-20,1975-11-14
204,318250.0,2036.0,ORANGE JUICE (FCOJ-A),1176,1975-12-01,1976-11-16


#### Then, for each commodity, we can query all the time series data for all of these relevant futures.

In [3]:
# we create a dictionary with the future codes as keys and their respective contract dates as values
futcodes_contrdates = info_df.set_index("futcode")["contrdate"].to_dict()
dict(list(futcodes_contrdates.items())[:5])

{46.0: '0303', 189.0: '0300', 190.0: '0704', 391.0: '0986', 912.0: '0188'}

In [4]:
# We query and fetch the time series data for the each of the future contracts
data_contracts = fetch_wrds_fut_contract(futcodes_contrdates, 'paper')
data_contracts

,futcode,date_,settlement,contrdate
0,46.0,2001-10-08,93.250000,0303
1,46.0,2001-10-09,93.250000,0303
2,46.0,2001-10-10,93.549988,0303
3,46.0,2001-10-11,93.799988,0303
4,46.0,2001-10-12,98.799988,0303
...,...,...,...,...
72477,318251.0,1977-11-10,124.500000,1177
72478,318251.0,1977-11-11,125.500000,1177
72479,318251.0,1977-11-14,129.850000,1177
72480,318251.0,1977-11-15,131.100000,1177


#### We now make some transformations to the futures time series data to allow for later calculations

In [5]:
# Convert the data from daily observations to monthly observations by keeping only the last observation of each month (for each contract)
monthly_df = futures_series_to_monthly(data_contracts)

In [6]:
monthly_df

,futcode,settlement,contr_period,obs_period
69174,318238.0,50.099991,1974-07,1973-08
69193,318238.0,51.599991,1974-07,1973-09
69214,318238.0,58.099991,1974-07,1973-10
70372,318243.0,57.799988,1974-09,1973-10
69234,318238.0,58.799988,1974-07,1973-11
...,...,...,...,...
59641,197774.0,112.150000,2008-11,2008-08
51657,161827.0,94.199997,2008-09,2008-09
59662,197774.0,89.199997,2008-11,2008-09
59685,197774.0,76.199997,2008-11,2008-10


In [7]:
# create a new dataframe in which there is only one observation per month, and each observation has the first through 12th contract settlements (relative to observation month)
first_through_12th_contracts_df = extract_first_through_12th_contracts(monthly_df)

This is necessary, since for the basis calculation, we need the one-month futures price and the 12-month futures price. But, not all futures have all 12 months traded. Within the paper's methodology, if the one-month is not available, we need the nearest. If the 12-month is not available, we use the latest (up to 12 months).

In [8]:
first_through_12th_contracts_df.iloc[30:40]

,1mth_settlement,2mth_settlement,3mth_settlement,4mth_settlement,5mth_settlement,6mth_settlement,7mth_settlement,8mth_settlement,9mth_settlement,10mth_settlement,11mth_settlement,12mth_settlement
1976-02,61.449997,NaN,63.349991,NaN,65.099991,NaN,66.599991,NaN,67.750000,NaN,68.799988,NaN
1976-03,NaN,62.599991,NaN,64.349991,NaN,65.799988,NaN,66.849991,NaN,67.799988,NaN,NaN
1976-04,59.449997,NaN,61.000000,NaN,62.500000,NaN,63.500000,NaN,63.849991,NaN,64.899994,NaN
1976-05,NaN,58.399994,NaN,59.199997,NaN,60.099991,NaN,60.949997,NaN,61.899994,NaN,NaN
1976-06,53.000000,NaN,53.299988,NaN,54.299988,NaN,55.199997,NaN,56.099991,NaN,57.299988,NaN
1976-07,NaN,53.449997,NaN,54.799988,NaN,55.549988,NaN,56.549988,NaN,57.799988,NaN,NaN
1976-08,48.000000,NaN,48.500000,NaN,49.750000,NaN,51.349991,NaN,52.849991,NaN,54.250000,NaN
1976-09,NaN,50.449997,NaN,51.399994,NaN,53.000000,NaN,54.399994,NaN,55.649994,NaN,NaN
1976-10,47.599991,NaN,47.949997,NaN,49.099991,NaN,50.199997,NaN,51.399994,NaN,52.599991,NaN
1976-11,NaN,46.949997,NaN,47.849991,NaN,49.000000,NaN,50.000000,NaN,51.149994,NaN,NaN


#### Now, we calculate the statistics for the Table 1.

We need to use the first_through_12th_contracts_df to calculate Basis, and the monthly_df to calculate the excess return statistics.

The basis is defined as:
$$ B_{i,t} = \frac{\log(F_{i,t,T_1}) - \log(F_{i,t,T_2})}{T_2 - T_1} $$



where:
- $ F_{i,t,T_1} $ is the one-month (or nearest) futures price ($ T_1 = 1 $)
- $ F_{i,t,T_2} $ is the 12-month (or latest up to 12-months) futures price ($ T_2 = 12 $)
- The basis is computed as the log difference between these prices, divided by the difference in maturity.


In [9]:
stats = compute_futures_stats(first_through_12th_contracts_df, monthly_df)

#### Key lines from this function:
``` python
basis_df['basis'] = (np.log(basis_df['settlement_T1']) - np.log(basis_df['settlement_T2'])) / basis_df['month_diff'] * 100

freq_bw = (basis_df["basis"] > 0).mean() * 100

excess_return_df = monthly_df.groupby("futcode").apply(
        lambda x: (x.sort_values(by="obs_period").iloc[-1]["settlement"] / x.sort_values(by="obs_period").iloc[0]["settlement"] - 1) * 100
    ).reset_index(name="excess_return")


In [14]:
stats

{'N': 419,
 'mean_basis': -0.308002088793494,
 'freq_bw': 34.60620525059666,
 'excess_return_mean': 4.1517311730356194,
 'excess_return_std': 39.04106471572015,
 'sharpe_ratio': 0.10634267285656011}

#### And we end up with our stats for one of our products!
#### Now let's look at how we do this for several products at once

In [15]:
product_list = [3160, 289, 3161] # example product list
summary_table = pd.DataFrame() # we create an empty dataframe to store the summary statistics of each product
sector_map = { # example sector map
    3160: "Agriculture",
    289: "Agriculture",
    3161: "Agriculture"}

for code in product_list: # we iterate through each product in the list
    row = process_single_product(code, 'paper') # we process the product (do the operations and calculations shown above)
    if row is not None:
        row["Sector"] = sector_map.get(code, "")
        summary_table = pd.concat([summary_table, row], ignore_index=True) # we add the product row to the summary table

summary_table

,Commodity,Contract Code,N,Basis,Freq. of Backwardation (%),E(Re) (Mean Annual Excess Return),σ(Re) (Std Dev of Excess Return),Sharpe Ratio,Sector
0,WESTERN BARLEY,3160.0,277,-0.368168,28.158845,5.231146,28.549440,0.183231,Agriculture
1,BUTTER (CASH),289.0,38,-0.222469,15.789474,-8.818858,10.262846,-0.859299,Agriculture
2,CANOLA,3161.0,342,-0.339598,29.532164,1.443846,23.135137,0.062409,Agriculture


#### Now let's create the table for all the products.
#### In essence, let's culminate our best replication of Table 1 in Investment shocks and the Commodity Basis Spread by Fan Yang

In [16]:
# we create tables for either the time period used in the paper, or the largest time period possible currently
table_paper = main_summary(time_period="paper")
table_current = main_summary(time_period="current")

In [18]:
final_paper = final_table(table_paper)
final_current = final_table(table_current)

print("=== PAPER PERIOD ===")
display(final_paper)
print("\n=== CURRENT PERIOD ===")
display(final_current)

=== PAPER PERIOD ===



=== CURRENT PERIOD ===
